In [1]:
import numpy as np
from sklearn import datasets
from numpy.random import MT19937
from numpy.random import RandomState, SeedSequence

In [2]:
#                    Description of the dataset
#  data : {ndarray, dataframe} of shape (150, 4)
#             The data matrix. If `as_frame=True`, `data` will be a pandas
#             DataFrame.
#         target: {ndarray, Series} of shape (150,)
#             The classification target. If `as_frame=True`, `target` will be
#             a pandas Series.
#         feature_names: list
#             The names of the dataset columns.
#         target_names: list
#             The names of target classes.
#         frame: DataFrame of shape (150, 5)
#             Only present when `as_frame=True`. DataFrame with `data` and
#             `target`.

In [3]:
# Creating an output array
def target_array(raw_target):
    target = []
    for i in range(len(raw_target)):
        if raw_target[i] == 0:
            target.append([1, 0, 0])
        elif raw_target[i] == 1:
            target.append([0, 1, 0])
        else:
            target.append([0, 0, 1])
    return np.array(target, dtype="uint8")
    

In [4]:
# Loading the Iris dataset
iris = datasets.load_iris()

data = iris.data
target = target_array(iris.target)

# Mixing data
# np.random.seed(1)
# np.random.shuffle(data)
# np.random.seed(1)
# np.random.shuffle(target)


rs1 = RandomState(MT19937(SeedSequence(123456789)))
rs1.shuffle(data)
rs2 = RandomState(MT19937(SeedSequence(123456789)))
rs2.shuffle(target)


In [5]:
# RELU
# Activation function
def activation_func(x): 
    z = (x > 0) * x     # returns x if x > 0
    return z            # return 0 otherwise


# Derivative of the activation function
def deriv_activation_func(x):
    z = x > 0
    return z            # returns 1 for input > 0

0 - input layer  
1 - hidden layer  
2 - output layer  
weights_1_to_2 - Weights linking the hidden layer and the output layer

In [6]:
# Neural network learning function
def neural_network_training(weights_0_to_1,
                            weights_1_to_2,
                            data,
                            target,
                            number_training_data,
                            epochs=5,
                            alpha=0.5):
    epoch_print_step = 100
    for epoch in range(epochs):
        if epoch % epoch_print_step == 0:
            print(f"\nepoch: {epoch + 1}")
        for i in range(number_training_data):
            # Forward propagation
            layer_0 = data[i:i + 1]
            layer_1 = activation_func(np.dot(layer_0, weights_0_to_1))
            layer_2 = np.dot(layer_1, weights_1_to_2)

            # Error calculation
            layer_2_delta = layer_2 - target[i:i + 1]
            layer_1_delta = layer_2_delta.dot(weights_1_to_2.T) * deriv_activation_func(layer_1)
            
            # Backpropagation
            weights_1_to_2 -= alpha * layer_1.T.dot(layer_2_delta)
            weights_0_to_1 -= alpha * layer_0.T.dot(layer_1_delta)
            
            # Output of a part of the data
            if epoch % epoch_print_step == 0 and i >= 10 and i <= 13:
                print(f"{np.argmax(layer_2) == np.argmax(target[i])} Error: {np.sum((layer_2 - target[i]) ** 2):.5f}, out = {layer_2}, target = {target[i]}")
  
    return weights_0_to_1, weights_1_to_2

In [7]:
# Declaring parameters for a neural network
# epochs = 1000
epochs = 201
alpha = 0.002

number_test_data = 30
len_data = len(data)
number_training_data = len_data - number_test_data
layer_1_size = 7

np.random.seed(1)
weights_0_to_1 = np.random.rand(len(data[0]), layer_1_size)
weights_1_to_2 = np.random.rand(layer_1_size, len(target[0]))

In [8]:
# Neural network training
weights_0_to_1, weights_1_to_2 = neural_network_training(weights_0_to_1,
                                                         weights_1_to_2,
                                                         data,
                                                         target,
                                                         number_training_data,
                                                         epochs,
                                                         alpha)


epoch: 1
True Error: 3.85272, out = [[1.40380717 1.83493981 1.08853956]], target = [0 1 0]
True Error: 1.12542, out = [[0.74288711 1.44316717 0.61411869]], target = [0 1 0]
False Error: 1.30621, out = [[0.40479657 0.94812762 0.23021712]], target = [1 0 0]
False Error: 6.12320, out = [[1.62512891 1.84752127 1.26233901]], target = [0 0 1]

epoch: 101
True Error: 0.00088, out = [[-0.01817917  1.010468    0.02106531]], target = [0 1 0]
True Error: 0.09958, out = [[ 0.12294345  0.7105184  -0.02585526]], target = [0 1 0]
True Error: 0.00303, out = [[ 0.97125593  0.04513733 -0.01284231]], target = [1 0 0]
True Error: 0.04945, out = [[-0.08275579  0.14175212  0.84996512]], target = [0 0 1]

epoch: 201
True Error: 0.00415, out = [[-0.0426412   1.04665585  0.01238989]], target = [0 1 0]
True Error: 0.09373, out = [[ 0.1289781   0.72395444 -0.0298366 ]], target = [0 1 0]
True Error: 0.00041, out = [[ 1.01340754  0.00902618 -0.01231756]], target = [1 0 0]
True Error: 0.03065, out = [[-0.06613946 

In [9]:
def сhecking_accuracy(data, target):    
    if np.argmax(data) == np.argmax(target):
        print("True")
        return
    print(f"False: {data}, {target}")

In [10]:
def neural_network_test(weights_0_to_1,
                        weights_1_to_2,
                        data,
                        target,
                        number_test_data,
                        len_data):
    # print(len_data - number_test_data, len_data)
    for i in range(len_data - number_test_data, len_data):
        layer_0 = data[i:i + 1]
        layer_1 = activation_func(np.dot(layer_0, weights_0_to_1))
        layer_2 = np.dot(layer_1, weights_1_to_2)
        # error = np.sum((layer_2 - target[i]) ** 2)
        
        сhecking_accuracy(layer_2, target[i])
        # print(f"target = out: {np.argmax(target[i]) == np.argmax(layer_2)}\terror = {error:.4f}")
        # print(f"target = {target[i]}, out = {layer_2},\terror = {error:.4f}")

In [11]:
neural_network_test(weights_0_to_1,
                    weights_1_to_2,
                    data,
                    target,
                    number_test_data,
                    len_data)

True
True
True
True
True
True
True
False: [[-0.02197351  0.27398408  0.6847794 ]], [0 1 0]
True
False: [[0.03292013 0.47738165 0.51623802]], [0 1 0]
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [12]:
def neural_network(weights_0_to_1,
                   weights_1_to_2,
                   data):
    layer_0 = data
    layer_1 = activation_func(np.dot(layer_0, weights_0_to_1))
    layer_2 = np.dot(layer_1, weights_1_to_2)    
    return layer_2

In [13]:
for i in range(20):
    number = np.random.randint(0, 150)
    prediction = neural_network(weights_0_to_1,
                                weights_1_to_2,
                                data[number])
    сhecking_accuracy(prediction, target[number])

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
